In [1]:
import pandas as pd
import re
import string

In [2]:
df_false_Arxiv = pd.read_csv('df_Fake_Arxiv.csv')
df_true_Arxiv = pd.read_csv('df_True_Arxiv.csv')
df_false_Kaggle = pd.read_csv('df_Fake_Kaggle.csv')
df_true_Kaggle = pd.read_csv('df_True_Kaggle.csv')
df_true_CBS = pd.read_csv('df_True_CBS.csv')

In [3]:
df_false_Arxiv.drop('Unnamed: 0', axis=1, inplace=True)
df_true_Arxiv.drop('Unnamed: 0', axis=1, inplace=True)
df_true_CBS.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
df_true_CBS['class'] = 1
df_true_Kaggle['class'] = 1
df_false_Kaggle['class'] = 0

In [5]:
df_false_Arxiv[['text', 'class']] = df_false_Arxiv[['content', 'label']]
df_true_Arxiv[['text', 'class']] = df_true_Arxiv[['content', 'label']]

In [6]:
df_false_Arxiv.drop(['content', 'label'], axis=1, inplace=True)
df_true_Arxiv.drop(['content', 'label'], axis=1, inplace=True)
df_true_CBS.drop('Link', axis=1, inplace=True)

In [7]:
df_true_Kaggle.drop(['subject', 'date', 'title'], axis=1, inplace=True)
df_false_Kaggle.drop(['subject', 'date', 'title'], axis=1, inplace=True)

In [8]:
df_false_Arxiv = df_false_Arxiv.sample(frac=1).reset_index(drop=True)
df_false_Arxiv = df_false_Arxiv[0:12469]

In [9]:
df_true = pd.concat([df_true_Kaggle, df_true_CBS, df_true_Arxiv], axis=0)
df_false = pd.concat([df_false_Kaggle, df_false_Arxiv], axis=0)

In [10]:
df_true = df_true.sample(frac=1).reset_index(drop=True)
df_false = df_false.sample(frac=1).reset_index(drop=True)

In [11]:
df_true

,text,class
0,London's Metropolitan Police said Saturday ni...,1
1,WASHINGTON/NEW YORK (Reuters) - U.S. President...,1
2,It was an eyebrow-raising moment she did not t...,1
3,WASHINGTON (Reuters) - President-elect Donald ...,1
4,"Belfast, Northern Ireland — Rioters set a hij...",1
...,...,...
33880,"Prince Harry, Duke of Sussex, and Meghan, Duch...",1
33881,Britain's biggest royal wedding in years -- th...,1
33882,(Reuters) - Former FBI Director James Comey de...,1
33883,George Clooney rented a private island for Ama...,1


In [12]:
df_false

,text,class
0,Every American needs to consider this new evid...,0
1,Trump has repeatedly decried the gun violence ...,0
2,If you ve subjected yourself to cable news ove...,0
3,Great news! Southern Methodist University in T...,0
4,I ve been a lifelong Republican all of my lif...,0
...,...,...
35945,"Most of the time, when a celebrity make a joke...",0
35946,The interactive transcript could not be loaded...,0
35947,"As of Tuesday morning, it seemed that Trump wa...",0
35948,This new effort to have companies report to th...,0


In [13]:
df_false['class'].value_counts()

class
0    35950
Name: count, dtype: int64

In [14]:
df_true['class'].value_counts()

class
1    33885
Name: count, dtype: int64

In [16]:
## REMOVE DUPLICATES
print('First Size of df_true: ', df_true.shape)
print('First Size of df_false: ', df_false.shape)
df_true.drop_duplicates(subset=['text'], inplace=True)
df_false.drop_duplicates(subset=['text'], inplace=True)
print('\nSize of df_true after removing duplicates: ', df_true.shape)
print('Size of df_false after removing duplicates: ', df_false.shape)
df_false = df_false.reset_index(drop=True)
df_true = df_true.reset_index(drop=True)

First Size of df_true:  (32032, 2)
First Size of df_false:  (28716, 2)

Size of df_true after removing duplicates:  (32032, 2)
Size of df_false after removing duplicates:  (28716, 2)


In [17]:
def remove_space(df, column, space):
    df[column] = df[column].apply(lambda x: x.replace(space, ' '))
    df[column] = df[column].apply(lambda x: x[1:] if x.startswith(' ') else x)

##DATA FOR MANUAL TESTING
manual_fake = df_false.tail(100)
for i in range(28715, 28615, -1):
    df_false.drop([i], axis=0, inplace=True)
    
manual_true = df_true.tail(100)
for i in range(32031, 31931, -1):
    df_true.drop([i], axis=0, inplace=True)
    
manual = pd.concat([manual_fake, manual_true], axis = 0)

##CONCATING THE TRUE AND FALSE DATASETS
df = pd.concat([df_true, df_false], axis=0)

##REMOVING THE NULL VALUES AND SPACES
null_values_df = df[df['text'] == ' '].index
df.drop(null_values_df, axis=0, inplace=True)

null_values_manual = manual[manual['text'] == ' '].index
manual.drop(null_values_manual, axis=0, inplace=True)

df['text'] = df['text'].apply(lambda x: str(x))
manual['text'] = manual['text'].apply(lambda x: str(x))

##REMOVING THE SPECIAL CHARACTERS
def word_drop(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('\\W', ' ', text)
    text = re.sub('https?://\S+|www\.\S+', ' ', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    
    return text

df['text'] = df['text'].apply(word_drop)
manual['text'] = manual['text'].apply(word_drop)

remove_space(df, 'text', '    ')
remove_space(df, 'text', '   ')
remove_space(df, 'text', '  ')
remove_space(manual, 'text', '    ')
remove_space(manual, 'text', '   ')
remove_space(manual, 'text', '  ')

df['text'] = df['text'].apply(lambda x: x.replace('u s', 'usa'))
manual['text'] = manual['text'].apply(lambda x: x.replace('u s', 'usa'))
df['text'] = df['text'].apply(lambda x: x.replace('u s a', 'usa'))
manual['text'] = manual['text'].apply(lambda x: x.replace('u s a', 'usa'))

df.dropna()
manual.dropna()

##RE-INDEXING
df = df.reset_index(drop=True)
manual = manual.reset_index(drop=True)

##CHECKING THE FINAL SIZE
print('\nLast Size of df: ', df.shape)
print('Last Size of manual: ', manual.shape)

##EXPORTING THE 'CLEANED' DATA
df.to_csv('dataset.csv')
manual.to_csv('manual_testing.csv')


Last Size of df:  (60544, 2)
Last Size of manual:  (200, 2)


In [18]:
df['class'].value_counts()

class
1    31930
0    28614
Name: count, dtype: int64

In [19]:
manual['class'].value_counts()

class
0    100
1    100
Name: count, dtype: int64

In [20]:
df

,text,class
0,london s metropolitan police said saturday nig...,1
1,washington new york reuters usa president dona...,1
2,it was an eyebrow raising moment she did not t...,1
3,washington reuters president elect donald trum...,1
4,belfast northern ireland rioters set a hijacke...,1
...,...,...
60539,build that wall and when mexico sends its peop...,0
60540,andy cohen has gotten an intimate look at luan...,0
60541,kansas secretary of state kris kobach has repo...,0
60542,get celebs updates directly to your inbox subs...,0


In [21]:
manual

,text,class
0,cnn hammered trump for retweeting white suprem...,0
1,donald trump s disgraced national security adv...,0
2,donald trump went on a whining rant about the ...,0
3,david daleiden the mastermind behind the debun...,0
4,the koch brothers arranged their meeting of we...,0
...,...,...
195,prince harry duke of sussex and meghan duchess...,1
196,britain s biggest royal wedding in years the u...,1
197,reuters former fbi director james comey detail...,1
198,george clooney rented a private island for ama...,1
